In [23]:
%pwd 

'/home/verge/Desktop/docker_test/backend/notebook'

In [25]:
%cd ..

/home/verge/Desktop/docker_test/backend


/home/verge/miniconda3/envs/rag/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableMap
from langchain_core.documents import Document

from IPython.display import Markdown, display

In [16]:
from langchain_ollama import ChatOllama, OllamaEmbeddings


ollama_llm = ChatOllama(
    model= 'llama3.2-vision',
    base_url="http://localhost:11434",
    temperature=0.7,
    num_ctx = 1024, # size of the context window
    num_predict = -2, # Maximum number of tokens to predict when generating text.
    top_k=40,
    top_p=0.9,  
)

ollama_embedding = OllamaEmbeddings(
    model="llama3.2-vision",
    base_url="http://localhost:11434"
)

In [21]:
# ollama_llm.invoke('what is ai').content
# ollama_embedding.embed_query('what is ai')

In [27]:
# for directory loader 


loader = PyPDFLoader('data/B13 - Chapter II - Procedure  for proceeding Abroad.pdf')
documents = loader.load()
len(documents)

11

In [28]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(documents)
len(chunks)

15

In [30]:
from langchain_milvus import Milvus
# creating new database
vectorstore = Milvus.from_documents(
                documents=chunks,
                embedding= ollama_embedding,
                collection_name='collection',
                connection_args={'uri': 'database/milvustest.db'},
                drop_old=True,
            )


In [31]:
# # loading existing database
vectorstore = Milvus(
    ollama_embedding,
    connection_args={'uri': 'database/milvustest.db'},
    collection_name='collection',
)

In [32]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
# from langchain.retrievers import EnsembleRetriever
# from langchain_community.retrievers import BM25Retriever

# Define the search parameters
search_kwargs = {"k": 5, "ef": 10}  # Ensure ef >= k

# Create the individual retrievers with the search parameters
retriever = vectorstore.as_retriever(
        search_type="similarity", 
        search_kwargs=search_kwargs
    )


In [33]:
# Retrieve relevant documents/products
query = 'how to get maternity leaves'
r4 = retriever.invoke(query)
r4

[Document(metadata={'page': 1, 'pk': 455145358243135490, 'source': 'data/B13 - Chapter II - Procedure  for proceeding Abroad.pdf'}, page_content='HR - Executives’ Manual Page B 13 / 2\nCHAPTER-Il\nm POLICIES, PROCEDURES & RULES\nB-13 Procedure for proceedinci Abroad\nnoted that the NOC issued to an individual is for a one-time visit and for a\nparticular purpose and will not be valid for subsequent visits for which the NOC\nhas been initially granted. NOC for visiting friendly countries like Nepal, Bhutan\nvihere passport / visas are not required is also required to be obtained.\n3. AVAILING OF LEAVE BY EMPLOYEES WHO PROCEED ABROAD ON\nTEMPORARY DUTY / TRAINING / OTHERWISE\nPossession of a passport by an individual is not an authority to go to any foreign\ncountry. Employees desirous of proceeding abroad on leave or on termination\nof course / training / conference / seminar in a foreign country must obtain\nprior sanction / approval of the concerned Competent Authority. An employee\nm

In [35]:
# # Define the prompt template

prompt = ChatPromptTemplate.from_template("""                                
                Please analyze the following text and provide a summary or key insights based on its 
                content: {context} + {question}

                Question: {question}
                If there are no clear questions, please offer general information or commentary 
                related to the themes and topics presented.
            """)


# Define the pipeline
chain = (
        RunnableMap({
            "context": retriever,  # Use your retriever here
            "question": RunnablePassthrough()  # Pass
        })
        | prompt 
        | ollama_llm 
        | StrOutputParser() 
    )


In [ ]:
chain.invoke('what is ai')

'Based on the context of the provided documents, it appears that AI is not explicitly mentioned as a specific topic or concept being discussed.\n\nHowever, I can provide some general information and commentary on the themes and topics presented in the documents:\n\nThe documents seem to be part of an HR manual for Mazagon Dock Shipbuilders Limited, outlining procedures for employees to obtain "No Objection Certificates" (NOCs) for various purposes, including obtaining passports and proceeding abroad. The documents also mention administrative processes, clearance from Vigilance Department, and approval from high-level authorities.\n\nIf I were to infer a connection to AI, it might be related to the use of digital tools or automation in processing and managing these NOC applications, although there is no explicit reference to AI technologies such as machine learning or natural language processing.'

: 